In [ ]:
import boto3
import sagemaker
from sagemaker.sklearn.model import SKLearnModel
import time

In [ ]:
# Configuration
region = boto3.Session().region_name or 'us-east-1'
bucket = f"demo-bucket-smus-ml-{region}"
role = "arn:aws:iam::058264284947:role/service-role/AmazonSageMaker-ExecutionRole-20241115T121975"

# Use existing trained model
model_data = "s3://demo-bucket-smus-ml-us-east-1/model-artifacts/orchestrated-training-1761669076/output/model.tar.gz"
source_dir = f"s3://{bucket}/training_code.tar.gz"

print(f"Region: {region}")
print(f"Bucket: {bucket}")
print(f"Model: {model_data}")
print(f"Source: {source_dir}")

In [ ]:
# Create SKLearnModel with inference script
sklearn_model = SKLearnModel(
    model_data=model_data,
    role=role,
    entry_point='inference.py',
    source_dir=source_dir,
    framework_version='1.2-1',
    py_version='py3'
)

print("✅ Model created")

In [ ]:
# Create transformer
transformer = sklearn_model.transformer(
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f's3://{bucket}/inference-results-test/'
)

print("✅ Transformer created")

In [ ]:
# Run batch transform
transform_job_name = f"test-inference-{int(time.time())}"
print(f"Starting batch transform: {transform_job_name}")

transformer.transform(
    data=f's3://{bucket}/inference-data/',
    content_type='text/csv',
    split_type='Line',
    job_name=transform_job_name
)

print(f"✅ Batch transform completed")
print(f"Results: {transformer.output_path}")